## Libraries and Data 

In [1]:
import pandas as pd
from datetime import timedelta
import matplotlib.pyplot as plt

In [39]:
df = pd.read_csv('customer_data.csv').dropna()
df

,customer_id,revenue,most_recent_visit,number_of_orders,recency_days
0,22086,777,5/14/2006,9,232
1,2290,1555,9/8/2006,16,115
2,26377,336,11/19/2006,5,43
3,24650,1189,10/29/2006,12,64
4,12883,1229,12/9/2006,12,23
...,...,...,...,...,...
39994,3249,998,12/1/2006,10,31
39995,6686,771,6/28/2006,8,187
39996,16418,1016,7/31/2006,9,154
39997,9117,678,6/20/2006,7,195


## Data Preparation 

In [40]:
#Transform Date Variable
df['most_recent_visit'] = pd.to_datetime(df['most_recent_visit'])

#Get last date available
snapshot_date = df['most_recent_visit'].max() + timedelta(days = 1)
snapshot_date

Timestamp('2006-12-31 00:00:00')

In [41]:
#Monetary
df['Monetary'] = df.revenue/df.number_of_orders

In [42]:
#Changing Variable Names
df.rename(columns = {'number_of_orders': 'Frequency',
                    'recency_days' : 'Recency'},
                    inplace = True)

df.head()

,customer_id,revenue,most_recent_visit,Frequency,Recency,Monetary
0,22086,777,2006-05-14,9,232,86.333333
1,2290,1555,2006-09-08,16,115,97.187500
2,26377,336,2006-11-19,5,43,67.200000
3,24650,1189,2006-10-29,12,64,99.083333
4,12883,1229,2006-12-09,12,23,102.416667


In [43]:
#Dropping columns

df = df.drop(columns = ['revenue', 'most_recent_visit'] )

## RFM Preparation

In [44]:
#Create Frequency Groups
df['F'] = pd.qcut(x = df['Frequency'], q = 3, labels = range(1,4))
df['M'] = pd.qcut(x = df['Monetary'], q = 3, labels = range(1,4))
df['R'] = pd.qcut(x = df['Recency'], q = 3, labels = range(3, 0, -1))
df.head()

,customer_id,Frequency,Recency,Monetary,F,M,R
0,22086,9,232,86.333333,2,1,1
1,2290,16,115,97.187500,3,2,2
2,26377,5,43,67.200000,1,1,3
3,24650,12,64,99.083333,3,2,3
4,12883,12,23,102.416667,3,2,3


In [45]:
#RFM Score
df['RFM'] = df[['R', 'F', 'M']].sum(axis = 1)
df.head(2)

,customer_id,Frequency,Recency,Monetary,F,M,R,RFM
0,22086,9,232,86.333333,2,1,1,4
1,2290,16,115,97.187500,3,2,2,7


In [49]:
#Creating the RFM function
def rfm_segment(df):
    if df['RFM'] >= 8:
        return "SuperStar"
    elif((df['RFM'] >= 5) and (df['RFM'] < 8)):
        return "Future Champion"
    else:
        return 'Low Relevance' 

## RFM 

In [50]:
#Applying the RFM Function
df['RFM_level'] = df.apply(rfm_segment, axis = 1)
df.head()

,customer_id,Frequency,Recency,Monetary,F,M,R,RFM,RFM_level
0,22086,9,232,86.333333,2,1,1,4,Low Relevance
1,2290,16,115,97.187500,3,2,2,7,Future Champion
2,26377,5,43,67.200000,1,1,3,5,Future Champion
3,24650,12,64,99.083333,3,2,3,8,SuperStar
4,12883,12,23,102.416667,3,2,3,8,SuperStar


In [51]:
#Looking into the segments
df.groupby('RFM_level').agg({
    'Recency': 'mean',
    'Frequency': 'mean',
    'Monetary': ['mean', 'count']
})

Recency  Frequency    Monetary       
                       mean       mean        mean  count
RFM_level                                                
Future Champion  171.844545   9.835508   96.980906  26445
Low Relevance    306.598969   7.135673   78.525950   7179
SuperStar         80.124863  12.846431  108.268368   6375